In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pick data[0:20000] to run faster

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
data = data.drop("id", axis = 1)
data = data[0:20000]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(data)

# K means

In [ ]:
#Let's select the optimum number of clusters for k-means classification
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++',
                    max_iter = 400, n_init = 10, random_state = 0)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)
#Plotting the results onto a line graph to observe 'The elbow'
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Association')
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

In [ ]:
#Applying kmeans to the dataset / Creating the kmeans classifier
kmeans = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 500, 
                n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(x)

In [ ]:
import copy
data_kmeans = data.copy()
data_kmeans["label"] = y_kmeans
data_kmeans

In [ ]:
len(data_kmeans[data_kmeans.label == 0]) / len(data_kmeans[data_kmeans.label == 1])

In [ ]:
#Visualising the clusters
plt.scatter(x[y_kmeans == 0, 0:38], x[y_kmeans == 0, 37:75], s = 75, 
            c = 'red', label = '0')
plt.scatter(x[y_kmeans == 1, 0:38], x[y_kmeans == 1, 37:75], s = 75, 
            c = 'blue', label = '1')

#Plotting the centroids of the clusters
plt.scatter(kmeans.cluster_centers_[:, 0:38], 
            kmeans.cluster_centers_[:, 37:75], s = 100, 
            c = 'black', label = 'Centroide')

plt.legend()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
C = kmeans.cluster_centers_
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x[:, 0], x[:, 1], x[:, 2], c = y_kmeans)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c='#050505', s=1000)

In [ ]:
xs = x[:, 0]
ys = x[:, 1]
plt.scatter(xs, ys,  c = y_kmeans, alpha=0.5)
plt.scatter(C[:,0], C[:, 1], s = 150, c = 'red', label = 'centroid')

# DBSCAN

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
data = data.drop("id", axis = 1)
data = data[0:20000].values

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(data)
distances, indices = nbrs.kneighbors(data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

pick 60 for eps

In [ ]:
dbsc = DBSCAN(eps = 60, min_samples = 100).fit(data)

In [ ]:
labels = dbsc.labels_
core_samples = np.zeros_like(labels, dtype = bool)
core_samples[dbsc.core_sample_indices_] = True
unique_labels = np.unique(labels)
colors = plt.cm.Spectral(np.linspace(0,1, len(unique_labels)))
for (label, color) in zip(unique_labels, colors):
    class_member_mask = (labels == label)
    xy = data[class_member_mask & core_samples]
    plt.plot(xy[:,0],xy[:,1], 'o', markerfacecolor = color, markersize = 10)
    xy2 = data[class_member_mask & ~core_samples]
    plt.plot(xy2[:,0],xy2[:,1], 'o', markerfacecolor = color, markersize = 5)
plt.title("DBSCAN on TPS JUNE test data")